<a href="https://colab.research.google.com/github/Jakeh33/Planning-and-Decision/blob/main/Planning_and_Decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 规划算法

---
《Planning Algorithm》- Steven M. LaValle
- [online](http://lavalle.pl/planning/book.html)



# 基本资料介绍

## 前言绪论
- 运动策略库 [MSL](http://lavalle.pl/msl/)
- 相关材料 [PA](http://lavalle.pl/planning/)




### 1.1 规划（过程 $\to$ 结果）
- 机器人运动规划：通常忽略动力学和其他各种微分约束，主要关注物体的平动和转动
- 机器人轨迹规划：应用机器人运动规划算法的解及确定如何以遵守机械限制的方式沿着规划解移动
- 控制理论运动规划：构建非线性动力系统的输入，驱动系统从初始状态移动到目标位置
- 人工智能规划：更倾向于离散问题，解决类似魔方、拼图、积木等任务。通过定义有限行动集，应用于离散状态集合，通过给出相应的行动序列构造解
 


### 1.3 基本组成
- 状态：状态空间的大小（状态量个数）多数情况下很大导致无法显示表达
- 时间：规划问题设计随时间进行的决策序列
- 行动：规划产生对状态进行操作的行动（输入/控制）
- 初始和目标状态
- 准则：根据状态及行动编码一个规划的期望结果
  - 可行性：寻找能够到达目标状态的规划
  - 最优性：可行的基础上满足某种最优指标
- 规划：开环（时间序列），闭环（状态相关的函数），如果不能反馈状态，需要根据目前可用的信息推导状态（信息状态）

## 离散规划
最简单的规划问题，状态空间有限或可数无线，因此不需要几何模型或者微分方程来描述离散规划问题，且不考虑不确定性。即所有模型完全可知和可预测。

### 2.1 离散可行规划简介
- 可行规划问题可以被阐述为确定是否存在一个输入序列使得有限状态机最终报告一个所需状态。

#### 2.1.1 问题表述
- 所有可能的状态x的几个称为状态空间X，对于离线规划，重要的是集合可数
- 状态转移函数$x' = f(x,u)$，行动$u \in U(x)$作用于当前状态x后产生新状态x'
- 规划的目的就是找到一个有限行动序列将初始状态变换到目标状态
- 可以将状态和行动组合形成**状态转移图(state transition graph)**



#### 2.1.2 实例
例2.1 二维栅格运动
- 状态$x=(i,j)$，坐标位置
- 行动$U =\{(\pm 1,0),(0,\pm1)\}$
- 状态转移$f(x,u) = x+u$
- 如果增加阴影，表示障碍，即构建出迷宫

例2.2 魔方
- 状态空间：位置颜色的组合几何
- 行动：12个可能
- 任务：找到行动序列返回同色状态



### 2.2 可行规划的搜索
图搜索算法，能够跟踪已经访问过的状态并访问所有可达状态，并且无多余探索过程，递增构建搜索图$\mathcal G(V,E)$

基本步骤：
1. 初始化：E为空，V包含一些初始状态
2. 选择顶点：为扩展选择一个顶点$x_{cur}$（通过保持优先级队列）
3. 应用行动：得到一个新状态$x_{new}$
4. 向图中插入一条有向边：
5. 检查解：确定$\mathcal G$是否编码了一条从起点到终点的路径
6. 返回第2步


#### 2.2.1 一般前向搜索
- 未访问：初始时除初始状态外全部为未访问
- 不活动：已访问及下一个可能的状态均为不活动（对搜索不再产生贡献）
- 活动的(Q)：已经遇见的和一些未访问的相邻状态

通用伪代码模板
```
Q.insert(x_i) and mark x_i as visited
while Q is not empty do 
  Q.getFirst() -> x
  if x in X_goal:
    return Success
  for u in U(x):
    x' = f(x,u)
    ## 指针序列 x' -> x,构建规划序列 
    if x' not visited :
      mark x' as visited
      Q.insert(x')
    esle:
      Resolve duplicate x'
return Failure
```
- Q保存活动的状态，各种**搜索算法明显的区别在于采用不同的函数对Q进行排序**。
- 最简单的方式：先进先出队列
- 上述代码仅确定是否存在解，没有给出规划

#### 2.2.2 特殊前向搜索

##### 广度优先（breadth-first search）
- 先进先出队列
- 搜索边界一致扩大
- 保证发现的第一个解使用的步数最小


##### 深度优先（depth-first search）
- 后进先出的栈

##### Dijkstra算法
- 寻找图中单源最短路径
- 动态规划的一种特殊形式
- 假设图中的每条边都有一个相应的非负代价$l(e)$，是应用行动所付出的代价，可以用$l(x,u)$表示，代表从状态x应用行动u产生的代价。
- 优先级队列Q按照已付代价和C升序：对每个状态x，值$C^*(x)$为从初始状态x_i到x产生的最优已付代价。
- 重复遇到某状态x'，如果已在Q队列中，比较代价值C，如果更新后需要重新排序。
- 当x'从Q中移除，该状态变为不活动，可以确定最优$C^*$
- 为什么说队列第一个元素必然是最优？：
  - 由于元素在Q中是按照代价升序排列，而想要达到第一个元素的其他路径必将经过其他状态，而其他状态的代价值必然有更大的代价。
- 假设用Fibonacci堆来实现，运行时间$O(|V|lg|V|+|E|)，|V|,|E|$分别对应图论表示的顶点和边的数量


##### A*
- Dijkstra的扩展，通过对从给定状态到目标状态的代价进行启发式估计，试图减少所需探索状态的数量。
- C(x)是从$x_i \to x$的已付代价，G(x)表示从$x \to x_g$的尚需代价。
- 可用动态规划法递增地计算$C^*(x)$，但是无法提前知道真正的最优尚需代价
- 在某些应用场景中可以构建一个合理的估计代价，如迷宫
- 假设代价就是总步数，$(i,j) \to (i',j')$的估计值$|i'-i|+|j'-j|$，不考虑障碍物。
- 目的是计算一个尽可能接近最优尚需代价值的估计值，并且保证不大于最优尚需代价值，记为$\hat G^*(x)$
- 运行逻辑与Dijkstra基本一致，除了排序函数:根据$C^*(x')+\hat G^*(x')$，即从起点到终点的最优代价的估计值。
- 估计值越接近真值，越利于搜索，当$\hat G^* =0$时，退化为Dijkstra

##### 最佳优先（best-first search）
- 按照最优尚需代价的估计值进行排序
- 得到解不一定是最优的
- 能够加速运算，但是经常非常“贪婪”，会偏爱那些在搜索中"看起来好的"状态


##### 迭代深化
- 搜索树有大量的分支时（下一层比上一层有更多的顶点）
- 将深度优先搜索转为一种系统的搜索方法，寻找所有与起点距离在迭代值以内的状态
- 相比广度优先搜索在最坏情况下性能要略好
- 与A\*思路结合可以产生IDA\*算法

#### 2.2.3 其他方案

##### 后向搜索


```
Backward 
Q.insert(x_g) and mark x_g as visited
while Q is not empty:
  x' = Q.get_first()
  if x' = x_i:
    return Success
  for u^ in U^(-1)(x):
    x = f^(-1)(x',u^)
    if x not in visited:
      mark x as visited
      Q.insert(x)
    else:
      Resolve duplicate x
return Failure 
```





##### 双向搜索


```
BiDirectional
QI.insert(x_i) and mark x_i as visited
QG.insert(x_g) and mark x_g as visited
while QI not empty and QG not empty:
  if QI not empty:
    x ← QI.GetFirst()
    if x in visited from x_g:
      return SUCCESS
    forall u ∈ U(x):
      x′ ← f(x, u)
      if x′ not visited:
        Mark x′ as visited
        QI.insert(x′)
      else:
        Resolve duplicate x′
  if QG not empty:
    x′ ← QG.GetFirst()
    if x′in visited from x_i:
      return SUCCESS
    forall u^ ∈ U^(−1)(x'):
      x ← f^(−1)(x',u^)
      if x not visited:
        Mark x as visited
        QG.Insert(x)
      else:
        Resolve duplicate x
return FAILUR
```



### 2.3 离散最优规划
- 从三个方面进行扩展：
  - 用一个阶段索引来指示当前的规划步
  - 引入一个代价泛函，指示在执行过程中累积的代价
  - 引入一个终止，指示何时终止规划
- 几乎所有的问题都可以用最小化或者最大化来定义性能指标
- 新增符号表示：
  - $\pi_k$表示K步规划
  - K是一个规划的准确长度($u_1,...,u_k)$
  - L表示K步规划的代价泛函，F代表最后一个阶段$F=K+1$
  - $L(\pi_k)=\sum^k_{k=1} l(x_k,u_k)+l_F(x_F) $
  - $l_F$取决于是否达到目标，达到为0，否则为无穷
  - 通过定义$l_F$将可行性约束转换为直接地优化
  


#### 2.3.1 最优定长规划(固定长度最优路径）
- 最优性原则导致——值迭代算法(value iteration)
- 采用迭代方法在状态空间上计算最优尚需代价（或已付代价）函数，特殊情况下演变为Dijkstra算法。

##### 后向值迭代
- 从短的最优规划中产生长的最优规划的关键在于在X上构造最优尚需代价函数。
- 第$1 \to F$中的第k步，令$G^*_k$表示在最优规划执行过程中从$k \to F$的累积代价（后向）
- 第一次迭代时，$G^*_F=l_F(x_F)=G^*_F(f(x_K,u_K))$（由于没有行动，立即能够得到最后阶段的代价）
- 第二次迭代时，$G^*_K = \min_{u_K} \{l(x_K,u_K) +l_F(x_F)\}$，对每一个状态$x_K$进行计算，计算了从$K到F=K+1$的最优规划
- 根据流程得到递归表达式
 $$G^*_k(x_k) = \min_{u_k} \{l(x_k,u_k)+G^*_{k+1}(x_{k+1}) \}$$
- 这种方式需要保存每个阶段、每个状态下满足递归式最小化的行动，需要$O(K|X|)$存储空间


##### 例2.3 五状态最优规划
- 问题描述：
  - ![有向图](http://lavalle.pl/planning/img420.gif)
  - $X=\{a,b,c,d,e\},k=4,x_I =a,X_G=d$
  - $a\to a =2;a\to b =2;b\to c =1;b\to d =4;c\to a =1;c\to d =1;d\to c=1;
  d\to e =1$

- 尚需代价函数

|  | a | b | c | d | e |
|:--:|:--:|:--:|:--:|:--:|:--:|
|$G^*_5$|$\infty$|$\infty$|$\infty$|0|$\infty$|
|$G^*_4$|$\infty$|4|1|$\infty$|$\infty$|
|$G^*_3$|6|2|$\infty$|2|$\infty$|
|$G^*_2$|4|6|3|$\infty$|$\infty$|
|$G^*_1$|6|4|5|4|$\infty$|

- 简单说明
  - G5对应最目标d，代价0
  - G4对应一步能够到d的b和c，分别为4和1，此时d为无穷，因为d不存在自回环
  - G3对应能够两步到d，即能够一步到b和c的a，b，d
  - 以此类推，得到最终结果，由于初始状态为a，则定步长条件下的代价为6

##### 前向值迭代
